### amazon.com 데이터 연습
- https://nijianmo.github.io/amazon/index.html


In [1]:
import pandas as pd
import json
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Magazine_Subscriptions_5.json.gz')

In [3]:
df.shape

(2375, 12)

In [ ]:
df2 = pd.read_csv("Magazine_Subscriptions.csv", header=None)
df2.columns = ["itemId", "userId", "rating","timestamp"]
df2.head()

In [8]:
df2["timestamp"] = pd.to_datetime(df2['timestamp'],unit='s')
df2.head()

,itemId,userId,rating,timestamp,timestemp
0,B00005N7P0,AH2IFH762VY5U,5.0,2001-11-08,2001-11-08
1,B00005N7P0,AOSFI0JEYU4XM,5.0,2001-10-31,2001-10-31
2,B00005N7OJ,A3JPFWKS83R49V,3.0,2007-03-24,2007-03-24
3,B00005N7OJ,A19FKU6JZQ2ECJ,5.0,2006-11-10,2006-11-10
4,B00005N7P0,A25MDGOMZ2GALN,5.0,2014-07-14,2014-07-14


### 도서 추천

In [10]:
!unzip BX-Book-Ratings_utf8.zip
book  = pd.read_csv("BX-Book-Ratings_utf8.csv", sep=";")

Archive:  BX-Book-Ratings_utf8.zip
  inflating: BX-Book-Ratings_utf8.csv  


In [11]:
book.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [12]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163771 sha256=d364e3a756245cb1bbb3388f1f6cd8ba1ca4aafe3bd9eaaba779b0a370be9e56
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
#위의 자료로 추천모형을 만들어보세요
# train: test  = 7:3
# KNNBasic()를 사용하셔서 IBCF, UBCF, SVD()
# 테스트에 대해서 rmse, mae 값을 비교해보세요
# 메모리 부족 시-> 20000건만 사용하셔서...
# 35분까지 해보세요~

In [14]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   User-ID      1149780 non-null  int64  
 1   ISBN         1149780 non-null  object 
 2   Book-Rating  1149772 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 26.3+ MB


In [ ]:
book["Book-Rating"].value_counts()

In [ ]:
from surprise import Dataset, Reader, SVD, KNNBasic, accuracy
from surprise.model_selection import train_test_split

book = book[ book["Book-Rating">0]]
book = book.sample(n=20000) #random하게 20000행 선택

reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(book[["User-ID","ISBN","Book-Rating"]], reader)

trainset, testset = train_test_split(data, test_size=0.2)

svd = KNNBasic(sim_options={"name":"cosine", "user-based":False})#SVD() #KNNBasic(sim_options=....)
svd.fit(trainset)

pred = svd.test(testset)
accuracy.rmse(pred)
accuracy.mae(pred)

In [20]:
pred[0]

Prediction(uid=224435, iid='0449911535', r_ui=9.0, est=2.4825177980528816, details={'was_impossible': False})